In [1]:
import os
import numpy as np
import pandas as pd
import swifter
from sklearn.model_selection import train_test_split
import utils
import catboost
import pickle

In [15]:
def load_test_private_csv(path):
    from itertools import repeat
    converters = dict(zip(utils.FOI_COLUMNS, repeat(utils.parse_array)))
    types = dict(zip(utils.SIMPLE_FEATURE_COLUMNS, repeat(np.float32)))
    test = pd.read_csv(os.path.join(path, "test_private_v3_track_1.csv.gz"),
                       index_col="id", converters=converters,
                       dtype=types,
                       usecols=[utils.ID_COLUMN]+utils.SIMPLE_FEATURE_COLUMNS+utils.FOI_COLUMNS)
    return test

In [16]:
full_test = load_test_private_csv('./')

/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [18]:
closest_hits_features = full_test.swifter.apply(
    utils.find_closest_hit_per_station, result_type="expand", axis=1)

Pandas Apply: 100%|██████████| 1452188/1452188 [14:46<00:00, 1637.36it/s]


In [19]:
# cols = [col for col in utils.SIMPLE_FEATURE_COLUMNS if not (col.startswith('Lextra') | col.startswith('Mextra'))]
cols = [col for col in utils.SIMPLE_FEATURE_COLUMNS]
cols

['ncl[0]',
 'ncl[1]',
 'ncl[2]',
 'ncl[3]',
 'avg_cs[0]',
 'avg_cs[1]',
 'avg_cs[2]',
 'avg_cs[3]',
 'ndof',
 'MatchedHit_TYPE[0]',
 'MatchedHit_TYPE[1]',
 'MatchedHit_TYPE[2]',
 'MatchedHit_TYPE[3]',
 'MatchedHit_X[0]',
 'MatchedHit_X[1]',
 'MatchedHit_X[2]',
 'MatchedHit_X[3]',
 'MatchedHit_Y[0]',
 'MatchedHit_Y[1]',
 'MatchedHit_Y[2]',
 'MatchedHit_Y[3]',
 'MatchedHit_Z[0]',
 'MatchedHit_Z[1]',
 'MatchedHit_Z[2]',
 'MatchedHit_Z[3]',
 'MatchedHit_DX[0]',
 'MatchedHit_DX[1]',
 'MatchedHit_DX[2]',
 'MatchedHit_DX[3]',
 'MatchedHit_DY[0]',
 'MatchedHit_DY[1]',
 'MatchedHit_DY[2]',
 'MatchedHit_DY[3]',
 'MatchedHit_DZ[0]',
 'MatchedHit_DZ[1]',
 'MatchedHit_DZ[2]',
 'MatchedHit_DZ[3]',
 'MatchedHit_T[0]',
 'MatchedHit_T[1]',
 'MatchedHit_T[2]',
 'MatchedHit_T[3]',
 'MatchedHit_DT[0]',
 'MatchedHit_DT[1]',
 'MatchedHit_DT[2]',
 'MatchedHit_DT[3]',
 'Lextra_X[0]',
 'Lextra_X[1]',
 'Lextra_X[2]',
 'Lextra_X[3]',
 'Lextra_Y[0]',
 'Lextra_Y[1]',
 'Lextra_Y[2]',
 'Lextra_Y[3]',
 'NShared',
 'M

In [20]:
test_concat = pd.concat(
    [full_test.loc[:, cols],
     closest_hits_features], axis=1)

In [24]:
model = catboost.CatBoostClassifier()
model.load_model('./track_2_model.cbm')

In [25]:
predictions = model.predict_proba(test_concat.values)[:, 1]

In [26]:
pd.DataFrame(data={"prediction": predictions}, index=test_concat.index).to_csv(
    "sample_submission_private.csv", index_label=utils.ID_COLUMN)